In [1]:
import os
import pandas as pd
import requests

os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

In [2]:
gwas_meta = 'results/refs/gwas_catalog/gwas-catalog-v1.0.3-studies-r2022-02-02.tsv'
gwas_meta = pd.read_table(gwas_meta)

In [3]:
gwas_meta.head()

,DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,MAPPED_TRAIT,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY,SUMMARY STATS LOCATION,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI
0,2017-08-04,27989323,Ahola-Olli AV,2016-12-13,Am J Hum Genet,www.ncbi.nlm.nih.gov/pubmed/27989323,Genome-wide Association Study Identifies 27 Lo...,beta-nerve growth factor levels,"3,531 Finnish ancestry individuals",NaN,...,beta-nerve growth factor measurement,http://www.ebi.ac.uk/efo/EFO_0008035,GCST004421,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-08-04,27989323,Ahola-Olli AV,2016-12-13,Am J Hum Genet,www.ncbi.nlm.nih.gov/pubmed/27989323,Genome-wide Association Study Identifies 27 Lo...,Vascular endothelial growth factor levels,"7,118 Finnish ancestry individuals",NaN,...,vascular endothelial growth factor measurement,http://www.ebi.ac.uk/efo/EFO_0004762,GCST004422,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-08-04,27989323,Ahola-Olli AV,2016-12-13,Am J Hum Genet,www.ncbi.nlm.nih.gov/pubmed/27989323,Genome-wide Association Study Identifies 27 Lo...,CTACK levels,"3,631 Finnish ancestry individuals",NaN,...,chemokine (C-C motif) ligand 27 measurement,http://www.ebi.ac.uk/efo/EFO_0008082,GCST004420,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-08-04,27989323,Ahola-Olli AV,2016-12-13,Am J Hum Genet,www.ncbi.nlm.nih.gov/pubmed/27989323,Genome-wide Association Study Identifies 27 Lo...,Eotaxin levels,"8,153 Finnish ancestry individuals",NaN,...,eotaxin measurement,http://www.ebi.ac.uk/efo/EFO_0008122,GCST004460,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-08-04,27989323,Ahola-Olli AV,2016-12-13,Am J Hum Genet,www.ncbi.nlm.nih.gov/pubmed/27989323,Genome-wide Association Study Identifies 27 Lo...,Fibroblast growth factor basic levels,"7,565 Finnish ancestry individuals",NaN,...,fibroblast growth factor basic measurement,http://www.ebi.ac.uk/efo/EFO_0008130,GCST004459,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN


## Getting all the OLS Link ID/Disease Information

In [42]:
disease_info = []
for i in range(0, 37):
    
    split_fn = 'results/notebooks/gwas_catalog_disease_traits/processed/split.ols_id.{}.txt'
    split_fn = split_fn.format(i)
    split_info = open(split_fn).readlines()
    split_info = [x.strip().split() for x in split_info]
    disease_info.extend(split_info)


In [43]:
disease_index_dict = {k:v for k,v in disease_info}

In [44]:
len(disease_index_dict.keys())

3687

## Add the final disease_or_disorder column to the gwas_meta file

In [45]:
dd_list = []
for i, uris in enumerate(gwas_meta['MAPPED_TRAIT_URI']):
    
    # default dd will be other
    dd_list.append('other')
    
    # assign disease or disorder when possible
    if type(uris) == str:
        for uri in uris.split(','):
            uri = uri.strip()
            
            if uri in disease_index_dict:
                
                if disease_index_dict[uri] == 'disease':
                    dd_list[i] = ('disease')
                    break

                elif disease_index_dict[uri] == 'disorder':
                    dd_list[i] = ('disorder')


            

In [47]:
gwas_meta['disease_or_disorder'] = dd_list

In [48]:
gwas_meta[gwas_meta.disease_or_disorder.isin(['disease', 'disorder'])]

,DATE ADDED TO CATALOG,PUBMED ID,FIRST AUTHOR,DATE,JOURNAL,LINK,STUDY,DISEASE/TRAIT,INITIAL SAMPLE SIZE,REPLICATION SAMPLE SIZE,...,MAPPED_TRAIT_URI,STUDY ACCESSION,GENOTYPING TECHNOLOGY,SUMMARY STATS LOCATION,SUBMISSION DATE,STATISTICAL MODEL,BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT,MAPPED BACKGROUND TRAIT URI,disease_or_disorder
26,2017-08-17,28548082,Southam L,2017-05-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28548082,Whole genome sequencing and imputation in isol...,C-reactive protein levels,"up to 1,476 Mylopotamos (founder/genetic isola...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0004458,GCST004530,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
33,2017-08-17,28548082,Southam L,2017-05-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28548082,Whole genome sequencing and imputation in isol...,Hematocrit,"up to 1,476 Mylopotamos (founder/genetic isola...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0004348,GCST004535,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
38,2017-08-18,28548082,Southam L,2017-05-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28548082,Whole genome sequencing and imputation in isol...,Waist-hip ratio,"up to 1,476 Mylopotamos (founder/genetic isola...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0004343,GCST004555,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
43,2017-08-17,28548082,Southam L,2017-05-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28548082,Whole genome sequencing and imputation in isol...,Low density lipoprotein cholesterol levels,"up to 1,476 Mylopotamos (founder/genetic isola...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0004611,GCST004541,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
49,2017-08-18,28548082,Southam L,2017-05-26,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/28548082,Whole genome sequencing and imputation in isol...,Triglyceride levels,"up to 1,476 Mylopotamos (founder/genetic isola...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0004530,GCST004550,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33094,2019-06-21,31015401,Wu Y,2019-04-23,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/31015401,Genome-wide association study of medication-us...,Medication use (drugs for peptic ulcer and gas...,"53,137 European ancestry cases, 79,230 Europea...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0009923,GCST007922,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
33097,2019-06-21,31015401,Wu Y,2019-04-23,Nat Commun,www.ncbi.nlm.nih.gov/pubmed/31015401,Genome-wide association study of medication-us...,Medication use (vasodilators used in cardiac d...,"5,546 European ancestry cases, 237,113 Europea...",NaN,...,http://www.ebi.ac.uk/efo/EFO_0009926,GCST007925,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
33127,2020-07-28,32450446,Wang H,2020-04-29,Neurobiol Aging,www.ncbi.nlm.nih.gov/pubmed/32450446,Genome-wide interaction analysis of pathologic...,Neuritic plaques (SNP x SNP interaction),"1,316 individuals",NaN,...,http://www.ebi.ac.uk/efo/EFO_0006798,GCST010342,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease
33130,2020-07-28,32450446,Wang H,2020-04-29,Neurobiol Aging,www.ncbi.nlm.nih.gov/pubmed/32450446,Genome-wide interaction analysis of pathologic...,Total amyloid (SNP x SNP interaction),"1,276 individuals",NaN,...,http://www.ebi.ac.uk/efo/EFO_0005194,GCST010339,Genome-wide genotyping array,NaN,NaN,NaN,NaN,NaN,NaN,disease


In [49]:
new_gwas_fn = 'results/refs/gwas_catalog/gwas-catalog-v1.0.3-studies-r2022-02-02.withDDColumn.tsv'
gwas_meta.to_csv(new_gwas_fn, sep='\t', index=False)